In [ ]:
fetch = False
# fetch = True

In [ ]:
import os
cwd = os.getcwd()
if ((cwd[0:2] != "C:")&(fetch==True)):
    from google.colab import drive
    drive.mount('/content/gdrive',force_remount=True)
    %cd "/content/gdrive/My Drive/Projects"
    !git clone https://github.com/ai357060/modelF.git
    import sys
    root_dir="/content/gdrive/My Drive/Projects/modelF"
    sys.path.append(root_dir)
    %cd "/content/gdrive/My Drive/Projects/modelF/wm"
    !git fetch --all
    !git reset --hard origin/master

In [ ]:
import os
cwd = os.getcwd()
if ((cwd[0:2] != "C:")&(fetch==False)):
    from google.colab import drive
    drive.mount('/content/gdrive',force_remount=True)
    %cd "/content/gdrive/My Drive/Projects"
    import sys
    root_dir="/content/gdrive/My Drive/Projects/modelF"
    sys.path.append(root_dir)
    %cd "/content/gdrive/My Drive/Projects/modelF/wm"

In [ ]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [ ]:
import pandas as pd
import numpy as np
import datetime as dt
import importlib
import tradestats
import pandas as pd
from datetime import datetime
importlib.reload(tradestats)
from tradestats import *

In [ ]:
prices = loaddata_1D('AUDUSD_Candlestick_1_D_BID_03.08.2003-24.10.2021.csv')
pricesASK = loaddata_1D('AUDUSD_Candlestick_1_D_ASK_03.08.2003-24.10.2021.csv')
prices['openASK'] = pricesASK.open
prices['highASK'] = pricesASK.high
prices['lowASK'] = pricesASK.low
prices['closeASK'] = pricesASK.close

In [ ]:
len(prices)

In [ ]:
dictList = []
colFeat = []
keyList = []

maKey  = [3,5,7,30,35,40,45,60,80,100,120,140,160]
atrKey = [5,10,20]
rsiKey = [5,10,15,20]

maDict   = ma3(prices, maKey,3,5,7)
atrDict  = atr(prices, atrKey)
rsiDict  = rsi(prices,rsiKey)

dictList.append(maDict.df); colFeat.append('ma'); keyList.append(maKey) 
dictList.append(atrDict.df); colFeat.append('atr'); keyList.append(atrKey) 
dictList.append(rsiDict.df); colFeat.append('rsi'); keyList.append(rsiKey) 

tt = [-1]
sl = [0.7,1.0,1.3,1.6,1.9,2.2]
tp = [0.7,1.0,1.3,1.6,1.9,2.2]
tsl = [0,0.5,0.7,0.9]
# tsl = [0]

atrperiod = 'atr10atr_prev'
savetrades = False

In [ ]:
# prices.dropna(inplace=True)
masterFrame = pd.DataFrame(index=prices.index)
masterFrame[['id','year','month','day','date','weekday','open','high','low','close','openASK','highASK','lowASK','closeASK','volume']] = prices[['id','year','month','day','date','weekday','open','high','low','close','openASK','highASK','lowASK','closeASK','volume']]   

In [ ]:
for i in range(0,len(dictList)):#iteracja po funkcjach
    for j in keyList[i]: #iteracja po okresach
        for k in list(dictList[i][j]): #iteracja po kolumnach danej funkcji w okresie
            colID = colFeat[i] + str(j) + k
            masterFrame[colID] = dictList[i][j][k]

In [ ]:
# masterFrame.to_csv(sep=';',path_or_buf='../Data/masterFrame.csv',date_format="%Y-%m-%d",index = False,na_rep='')

In [ ]:
masterFrame = masterFrame.dropna()

In [ ]:
starttime = datetime.now()
# alltrades = preparetrades_brut_tsl(masterFrame,[1],[0.001,0.002,0.003,0.004,0.005,0.006],[0.001,0.002,0.003,0.004,0.005,0.006],[0.001,0.002,0.003,0.004,0.005,0.006],2015,2021)
# alltrades = preparetrades_brut_tp(masterFrame,[1],[0.003,0.004,0.005,0.006,0.007,0.008,0.009,0.01],[0.003,0.004,0.005,0.006,0.007,0.008,0.009,0.01],2003,2021)
# alltrades = preparetrades_brut_tp(masterFrame,[-1,1],[0.5],[0.9],2020,2021,atrperiod)
# alltrades = preparetrades_brut_tp(masterFrame,[-1],[0.3,0.4,0.5,0.6,0.7,0.8,0.9],[0.6,0.7,0.8,0.9,1,1.1,1.2,1.3,1.4,1.5],2003,2021,atrperiod)
# alltrades = preparetrades_brut_tp(masterFrame,[1],[1,1.2,1.4,1.6,1.8,2,2.2],[1.2,1.4,1.6,1.8,2,2.2,2.4,2.6,2.8,3,3.2],2003,2021,atrperiod)
alltrades = preparetrades_brut_tsl(masterFrame,tt,sl,tp,tsl,2003,2021,atrperiod)
# alltrades = preparetrades_brut_tsl(masterFrame,[1],sl,tp,tsl,2003,2021,atrperiod)
# alltrades = preparetrades_brut_tp(masterFrame,[-1],[0.003,0.004,0.005,0.006,0.007,0.008,0.009,0.01],[0.006,0.007,0.008,0.009,0.01,0.011,0.012,0.013,0.014,0.015],2003,2021,atrperiod)
# alltrades = preparetrades_brut_tp(masterFrame,[-1],[0.003,0.004,0.005,0.006,0.007,0.008,0.01,0.012,0.014],[0.003,0.004,0.005,0.006,0.007,0.008,0.01,0.012,0.014],2003,2021,atrperiod)
endtime = datetime.now()
print(str(endtime - starttime))

In [ ]:
trades = cleartrades_brut(alltrades.copy(),savetrades)

In [ ]:
def runstats(alltrades,ma1,atrperiod,tt,sl,tp,tsl,ma2t,rsit):
    for i in range(0,len(ma2t)):
        for j in range(0,len(rsit)):
            ma2=str(ma2t[i])
            rsi=str(rsit[j])            
            stats = runstats_ma(alltrades,'ma'+ma1+'SMAdiffseq_prev', 'ma'+ma1+'SMAvs'+ma2+'_prev', 'ma'+ma2+'SMAdiffseq_prev', 'rsi'+rsi+'rsidiffseq_prev', 'rsi'+rsi+'rsi_prev', atrperiod,tt,sl,tp,tsl,'x1')

    return stats

def runstats_ma(alltrades,a,sv,aa,aaa,bbb,atr,tt=[],sl=[],tp=[],tsl=[],ff=''):
    conf   = {}
    params = {}
    
    params['tradetype'] = [2,tt]
    params['sl'] =        [2,sl]
    params['tp'] =        [2,tp]
    params['tsl'] =       [2,tsl]
    params[atr]  =        [3,[-1000],[0.015]]
    params[a]    =        [1,[1,2,3,4]]
    params[sv]    =       [0,[-1000],[1000]]
    params[aa]   =        [1,[1,2,3,4]]
    params[aaa]    =      [3,[-1000,0,-1000],[0,1000,1000]]
    params[bbb]    =      [0,[70,60,50,40,30,0],[30,40,50,60,70,100]]

    conf['filename'] =    'ma_54_2003_2021_1_'+atr+'_'+sv+bbb
    print(conf['filename'])
    stats = stathyperparams2(alltrades,params,conf)
    return stats

In [ ]:
# runstats(alltrades,'30',atrperiod,tt,sl,tp,tsl,[3,5],[15])
# runstats(alltrades,'35',atrperiod,tt,sl,tp,tsl,[3,5],[15])
# runstats(alltrades,'40',atrperiod,tt,sl,tp,tsl,[3,5],[15])
# runstats(alltrades,'45',atrperiod,tt,sl,tp,tsl,[3,5],[15])
# runstats(alltrades,'60',atrperiod,tt,sl,tp,tsl,[3,5],[15])
# runstats(alltrades,'80',atrperiod,tt,sl,tp,tsl,[3,5],[15])
# runstats(alltrades,'100',atrperiod,tt,sl,tp,tsl,[3,5],[15])
# runstats(alltrades,'120',atrperiod,tt,sl,tp,tsl,[3,5],[15])
# runstats(alltrades,'140',atrperiod,tt,sl,tp,tsl,[3,5],[15])
# runstats(alltrades,'160',atrperiod,tt,sl,tp,tsl,[3,5],[15])

    

# [3,5,7,30,35,40,45,60,80,100,120,140,160]


In [ ]:
# fxs = [
# '{"tradetype": [2, 1], "sl": [2, 0.65], "tp": [2, 0.75], "tsl": [2, 0], "atr20atr_prev": [3, -1000, 0.015], "ma40SMAdiffseq_prev": [0, 3, 1000], "ma40SMAvs5_prev": [0, 0, 1000], "ma5SMAdiffdiff_prev": [0, -1000, 0], "rsi15rsidiffseq_prev": [0, -5, 3], "rsi15rsi_prev": [0, 30, 100]}'
# ]    
# sub = calcandplot(alltrades,fxs) 